In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

## Build the CNN model

In [20]:
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (3, 3), stride = (1,1), padding = (1, 1))
        self.pool = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
        
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (3, 3), stride = (1,1), padding = (1, 1))
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)

        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x

## Set device

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

## Hyper parameters

In [16]:
in_channels = 1
out_channels = 8
num_classes = 10
learning_rate = 0.001
batch_size = 64
epochs = 3

## Load the data

In [5]:
transform = transforms.ToTensor() 

# 1. Training Dataset
train_dataset = datasets.MNIST(
    root='./data', 
    train=True, 
    transform=transform, 
    download=True
)

# 2. Test Dataset
test_dataset = datasets.MNIST(
    root='./data', 
    train=False, 
    transform=transform, 
    download=True 
)

100%|██████████| 9.91M/9.91M [00:00<00:00, 12.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 367kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.40MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.26MB/s]


In [6]:
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=True)

## Initialize the network

In [21]:
model = CNN().to(device)

## Loss and Optimizer

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

## Train the network

In [23]:
for epoch in range(epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device = device)
        targets = targets.to(device = device)

        # Farward pass
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient Descent
        optimizer.step() 

## Check accuracy on tarining and test, to look how good the model is performing

In [24]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on testing data")
        
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device = device)
            y = y.to(device = device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

In [25]:
model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 58888 / 60000 with accuracy 98.15
Checking accuracy on testing data
Got 9790 / 10000 with accuracy 97.90
